# Explore GRS images

## Load python libraries

In [ ]:

import glob
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as xrio # activate the rio accessor

#import cartopy
import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import locale
#locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.use('TkAgg')
import hvplot.xarray

import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews import opts

import datashader as ds
from datashader import transfer_functions as tf 
import colorcet as cc
import panel as pn
import panel.widgets as pnw
#import ipywidgets as widgets

import pyproj as ppj
import rasterio
from affine import Affine
from shapely.geometry import box,Point, mapping
from shapely.ops import transform


import grstbx
from grstbx import visual

u = grstbx.utils
opj = os.path.join
hv.extension('bokeh')
grstbx.__version__

## Set Dask local cluster

In [ ]:
#import dask.distributed
#cluster = dask.distributed.LocalCluster()
#client = dask.distributed.Client(cluster)
#cluster.dashboard_link

## Set PROJ path if necessary

In [ ]:
ppj.datadir.get_data_dir()
#ppj.datadir.set_data_dir('/work/scratch/harmelt/envs/grstbx/share/proj')

## Set the images you want to play with

In [ ]:
odir ='/home/harmel/Dropbox/satellite/S2/cnes/datacube/'


basename='31TGM_2022-01-01_2022-06-30_SHL2'
ofile = opj(odir,basename+'.nc')

# Open and load your datacube:
# from netcdf format and interpret coordinate system
raster = xr.open_dataset(ofile,decode_coords='all')
dc =grstbx.l2grs()
dc.raster=raster


## **Fast checking of the RGB images**

In [ ]:
bands=[4,2,1]
#bands=[3,2,1]
coarsening = 2
brightness_factor = 5
gamma=2
fig = (dc.raster.Rrs.isel(wl=bands)[:,:,::coarsening, ::coarsening]**(1/gamma)*brightness_factor).plot.imshow(col='time', col_wrap=4,robust=True)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


## **Check spectral datacube** (i.e., Remote Sensing Reflectance, R<sub>rs</sub>, sr<sup>-1</sup>)

To quickly check your data visually, you can use the *visual* module of *grstbx*

In [ ]:
v=visual.view_spectral(dc.raster.Rrs)

In [ ]:
v.visu()


## If you want to clip your raster for a specific (hand-drawn) region of interest

In [ ]:
v.aoi_stream.data

In [ ]:
v.aoi_stream.element.geom()

In [ ]:
from shapely.geometry import Polygon
geom = v.aoi_stream.data
ys,xs=geom['ys'][-1],geom['xs'][-1]
polygon_geom = Polygon(zip(xs, ys))
polygon = gpd.GeoDataFrame(index=[0], crs=3857, geometry=[polygon_geom]) 
geom_ = polygon.to_crs(3857)#Rrs_masked.rio.crs) #4326)#32631)#4326)
geom_.to_crs(4326).bounds

# if you want to subset within the box (defined from the polygon) uncomment the following lines
#minx, miny, maxx, maxy = geom_.bounds.values[0]
#Rrs_masked.sel(x=slice(minx, maxx), y=slice(maxy, miny))

# if projected uncomment
#geom_=geom_.to_crs(Rrs_masked.rio.crs)


In [ ]:
Rrs_clipped = dc.raster.Rrs.rio.clip(geom_.geometry.values)

## If you do not want to clip your raster, just copy it into the Rrs_clipped variable

In [ ]:
Rrs_clipped = dc.raster.Rrs

## Now you can check the Rrs time series

In [ ]:
stacked = Rrs_clipped.dropna('time',thresh=0).stack(gridcell=["y", "x"]).dropna('gridcell',thresh=0)


In [ ]:
group_coord ='wl'
stat_coord='gridcell'
stats = xr.Dataset({'median':stacked.groupby(group_coord).median(stat_coord)})
stats['q25'] = stacked.groupby(group_coord).quantile(0.25,dim=stat_coord)
stats['q75'] = stacked.groupby(group_coord).quantile(0.75,dim=stat_coord)
stats['min'] = stacked.groupby(group_coord).min(stat_coord)
stats['max'] = stacked.groupby(group_coord).max(stat_coord)
stats['mean'] = stacked.groupby(group_coord).mean(stat_coord)
stats['std'] = stacked.groupby(group_coord).std(stat_coord)
stats['pix_num'] = stacked.count(stat_coord)

In [ ]:
%matplotlib inline
num_items = len(stats.time)
col_wrap=4
rows=int(np.ceil(num_items/col_wrap))

fig, axs = plt.subplots(nrows=rows,ncols=col_wrap, sharex=True,sharey=True, figsize=(20, rows*3.5))#,sharey=True
fig.subplots_adjust(hspace=0.1,wspace=0.1)
axs_ = axs.ravel()
[axi.set_axis_off() for axi in axs_]
for iax,(_,group) in enumerate(stats.groupby('time')): 
    date = group.time.dt.date.values
    
    axs_[iax].set_axis_on()
    axs_[iax].axhline(y=0,color='k',lw=1)
    axs_[iax].plot(group.wl,group['median'],c='k')
    axs_[iax].plot(group.wl,group['mean'],c='red',ls='--')
    axs_[iax].fill_between(group.wl, group['q25'], group['q75'],alpha=0.3,color='grey')
    axs_[iax].set_title(date)
plt.show()

# Check surface rugosity via sunglint BRDF

In [ ]:

coarsening = 2
fig=dc.raster.BRDFg[:,::coarsening, ::coarsening].plot(col='time', col_wrap=4,vmax=0.01,robust=True,cmap=cc.cm.gray)#,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig

# Apply L2B algorithms

In [ ]:
#simplify name of dc.raster.Rrs or Rrs_clipped
Rrs = dc.raster.Rrs

## Check blue over green ratio for Chl retrieval with OC2 from NASA
$log_{10}(chlor\_a) = a_0 + \sum\limits_{i=1}^4 a_i \left(log_{10}\left(\frac{R_{rs}(\lambda_{blue})}{R_{rs}(\lambda_{green})}\right)\right)^i$

In [ ]:
# NASA OC2 fro MODIS; bands 488, 547 nm
a = [0.2500,-2.4752,1.4061,-2.8233,0.5405]
# NASA OC2 for OCTS; bands 490, 565 nm
a = [0.2236,-1.8296,1.9094,-2.9481,-0.1718]

ratio = np.log10(Rrs.isel(wl=1)/Rrs.isel(wl=2))
logchl=0
for i in range(len(a)):
    logchl+=a[i]*ratio**i
chl = 10**(logchl)
chl.name='chl in mg.m-3 from OC2'

Set range of valid values

In [ ]:
chl = chl.where((chl >= 0) & (chl <= 80))
#chl.persist()



In [ ]:
coarsening=1
fig=chl[:,::coarsening, ::coarsening].plot(col='time', col_wrap=4,robust=True,cmap=cc.cm.CET_D13)#,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig

# CDOM retrieval based on Brezonik et al, 2015


In [ ]:
a = [1.872,-0.83]
acdom = np.exp(a[0] + a[1] * np.log(Rrs.isel(wl=1)/Rrs.isel(wl=5)))
acdom.name='CDOM absorption at 440 nm-1'
acdom= acdom.where((acdom >= 0) & (acdom <= 10))
#acdom.persist()

In [ ]:
fig=acdom[:,::coarsening, ::coarsening].plot(col='time', col_wrap=4,robust=True,cmap=cc.cm.CET_D13)#,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig

# Total suspended particulate matter (SPM) from Nechad et al., 2010, 2016 formulation
spm in mg/L

In [ ]:
a = [610.94*np.pi, 0.2324/np.pi]
Rrs_ = Rrs.isel(wl=3)
spm = a[0] * Rrs_ / (1 - ( Rrs_/ a[1]))
spm.name='CDOM absorption at 440 nm-1'
spm= spm.where((spm >= 0) & (spm <= 150))
#spm.persist()

In [ ]:
fig=spm[:,::coarsening, ::coarsening].plot(col='time', col_wrap=4,robust=True,cmap=cc.cm.bky)#,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


# Play with time series

In [ ]:

def compute_stats(xarr, stat_coord='gridcell',limits=[-np.inf,np.inf],dropna=True,time_coord='time'):
    xarr = xarr.where((xarr>limits[0])&(xarr<limits[1]))
    stats = xr.Dataset({'median':xarr.median(stat_coord)})
    stats['q25'] = xarr.quantile(0.25,dim=stat_coord)
    stats['q75'] = xarr.quantile(0.75,dim=stat_coord)
    stats['min'] = xarr.min(stat_coord)
    stats['max'] = xarr.max(stat_coord)
    stats['mean'] = xarr.mean(stat_coord)
    stats['std'] = xarr.std(stat_coord)
    stats['pix_num'] = xarr.count(stat_coord)
    if dropna:
        return stats.dropna(time_coord)
    else:
        return stats


In [ ]:
stacked = chl.stack(gridcell=["y", "x"]).dropna('gridcell','all')
stats = compute_stats(stacked,limits=[0.01,100])


In [ ]:
df = stats.to_dataframe()#.reset_index()
pn.widgets.DataFrame(df,height=300,width=1200)

In [ ]:
import pandas_bokeh
df = stats.to_dataframe()
df[['q25','median','q75']].plot_bokeh(ylabel=param,figsize=(1000, 500),plot_data_points=True,colormap=["grey", "black","grey"],rangetool=True)